In [1]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, Subset
from transformers import AutoFeatureExtractor, AutoModelForAudioClassification 
import pandas as pd
import librosa
import numpy as np
from tqdm import tqdm

import sys
sys.path.append("..")
from src.models import EModel, Wav2Vec2Facebook

In [2]:
model_name_or_path = "facebook/wav2vec2-base"
feature_extractor = AutoFeatureExtractor.from_pretrained(model_name_or_path)
sampling_rate = feature_extractor.sampling_rate

path = "/home/work/joono/joono/joono/DV_DV.Deep/w2cs5agc/checkpoints/best-checkpoint_oneshot.ckpt"
model = Wav2Vec2Facebook.load_from_checkpoint(path, args={})

/home/work/joono/anaconda3/envs/torch/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/home/work/joono/anaconda3/envs/torch/lib/python3.12/site-packages/transformers/configuration_utils.py:364: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(
Some weights of Wav2Vec2ForSequenceClassification were not initialized from the model checkpoint at facebook/wav2vec2-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'projector.bias', 'projector.weight', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizat

======== Trainable Lora layers ===========
torch.Size([8, 768])
torch.Size([768, 8])
torch.Size([8, 768])
torch.Size([768, 8])
torch.Size([8, 768])
torch.Size([768, 8])
torch.Size([8, 768])
torch.Size([768, 8])
torch.Size([8, 768])
torch.Size([768, 8])
torch.Size([8, 768])
torch.Size([768, 8])
torch.Size([8, 768])
torch.Size([768, 8])
torch.Size([8, 768])
torch.Size([768, 8])
torch.Size([8, 768])
torch.Size([768, 8])
torch.Size([8, 768])
torch.Size([768, 8])
torch.Size([8, 768])
torch.Size([768, 8])
torch.Size([8, 768])
torch.Size([768, 8])
torch.Size([8, 768])
torch.Size([768, 8])
torch.Size([8, 768])
torch.Size([768, 8])
torch.Size([8, 768])
torch.Size([768, 8])
torch.Size([8, 768])
torch.Size([768, 8])
torch.Size([8, 768])
torch.Size([768, 8])
torch.Size([8, 768])
torch.Size([768, 8])
torch.Size([8, 768])
torch.Size([768, 8])
torch.Size([8, 768])
torch.Size([768, 8])
torch.Size([8, 768])
torch.Size([768, 8])
torch.Size([8, 768])
torch.Size([768, 8])
torch.Size([8, 768])
torch.Size([

In [3]:
def speech_file_to_array_fn(path):
    audio, _ = librosa.load(path, sr=sampling_rate)
    inputs = feature_extractor(audio, sampling_rate=sampling_rate, return_tensors="pt", padding=True)
    return inputs.input_values.squeeze()

In [4]:
# Collate 함수 정의
def collate_fn(batch):
    signals, labels = zip(*batch)
    max_length = max([signal.size(0) for signal in signals])
    padded_signals = torch.zeros(len(signals), max_length)
    for i, signal in enumerate(signals):
        padded_signals[i, :signal.size(0)] = signal
    labels = torch.tensor(labels)
    return padded_signals, labels

In [5]:
class TestDataset(Dataset):
    def __init__(self, df):
        self.df = df

    def __len__(self):
        return self.df.shape[0]
    
    def __getitem__(self, idx):
        path = os.path.join("..", "dataset", self.df.loc[idx, 'path'])
        if not os.path.exists(path):
            raise FileNotFoundError(f"파일을 찾을 수 없습니다: {path}")     
        signal = speech_file_to_array_fn(path)           
        return signal, -1

In [6]:
test_df = pd.read_csv('../dataset/test.csv', index_col=None)
test_df['path'] = '../dataset/' + test_df['path'].str[1:]
test_dataset = TestDataset(test_df)
test_loader = DataLoader(test_dataset, shuffle=False, num_workers=24, batch_size=32, collate_fn=collate_fn)

In [7]:
def inference(model, test_loader, device):
    model.to(device)
    model = model.eval()
    predictions = []
    with torch.no_grad():
        for inputs, labels in tqdm(test_loader):
            
            inputs = inputs.to(device)
            labels = labels.to(device)
            
            probs = model(inputs)

            probs  = probs.cpu().detach().numpy()
            predictions += probs.tolist()
            
    return predictions

In [8]:
preds = inference(model=model, test_loader=test_loader, device='cuda:0')
# preds = model.inference(test_loader=test_loader)

  0%|          | 0/1563 [00:00<?, ?it/s]/home/work/joono/anaconda3/envs/torch/lib/python3.12/site-packages/torch/nn/modules/conv.py:306: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at /opt/conda/conda-bld/pytorch_1712608847532/work/aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return F.conv1d(input, weight, bias, self.stride,
100%|██████████| 1563/1563 [00:49<00:00, 31.38it/s]


In [9]:
submit = pd.read_csv('/home/work/joono/joono/dataset/sample_submission.csv')

max_thres = 0.999
min_thres = 0.001

for i in tqdm(range(len(preds))):
    if      preds[i][0] > max_thres : submit.iloc[i, 1] = 1
    elif    preds[i][0] < min_thres : submit.iloc[i, 1] = 0 
    else                            : submit.iloc[i, 1] = preds[i][0]
    if      preds[i][1] > max_thres : submit.iloc[i, 2] = 1
    elif    preds[i][1] < min_thres : submit.iloc[i, 2] = 0 
    else                            : submit.iloc[i, 2] = preds[i][1]
submit.head()

  0%|          | 0/50000 [00:00<?, ?it/s]/tmp/ipykernel_3057227/951011515.py:9: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.06928792595863342' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  else                            : submit.iloc[i, 1] = preds[i][0]
/tmp/ipykernel_3057227/951011515.py:12: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.9303889274597168' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  else                            : submit.iloc[i, 2] = preds[i][1]
100%|██████████| 50000/50000 [00:08<00:00, 6147.75it/s]


,id,fake,real
0,TEST_00000,0.069288,0.930389
1,TEST_00001,0.078578,0.920674
2,TEST_00002,0.072870,0.924842
3,TEST_00003,0.061994,0.943236
4,TEST_00004,0.090725,0.906439


In [10]:
submit.to_csv('joono_wav2vec2_test_submit.csv', index=False)

In [ ]:
preds